In [ ]:
!pip install ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 66.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [ ]:
from ultralytics import YOLO
from roboflow import Roboflow

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
rf = Roboflow(api_key="bXpO1DKd0W283XJpKM3h")
project = rf.workspace("echs-pmipl").project("neu-surface-defect-pohqa-mmnnu")
dataset = project.version(1).download("yolov8")
model = YOLO('yolov8s.pt')

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to neu-surface-defect-1 in yolov8:: 100%|██████████| 3610/3610 [00:00<00:00, 13093.24it/s]


In [ ]:

results = model.train(
    data='/content/neu-surface-defect-1/data.yaml',  # Dataset path
    epochs=150,              # Extended training for full convergence
    imgsz=640,               # Image resolution (768 if GPU allows)
    batch=16,                # Adjust to your GPU memory (set -1 for auto)
    patience=20,             # Early stopping patience
    lr0=0.001,               # Initial learning rate
    cos_lr=True,             # Cosine LR schedule for smoother convergence
    optimizer='AdamW',       # Stable optimizer for small datasets
    weight_decay=0.0005,     # Regularization to reduce overfitting
    save=True,               # Save best.pt automatically
    project='quality_control',
    name='defect_detection_v1',

    # 🔸 Data augmentation for subtle metal defects
    hsv_h=0.015,             # Minimal color hue variation
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,             # No rotation (orientation fixed)
    flipud=0.0,              # Avoid vertical flip if gravity matters
    fliplr=0.5,              # Horizontal flip (ok for symmetry)
    mosaic=1.0,              # Combine multiple images
    mixup=0.1,               # Mild blending between samples
    scale=0.1,               # Small random scaling
    copy_paste=0.3,          # Boost rare defect classes
    augment=True,            # Enable augmentations

    # ⚙️ Logging and evaluation
    val=True,                # Validate after each epoch
    plots=True,              # Save PR curves, confusion matrix, etc.
    verbose=True,            # Show detailed logs
    device=0                 # GPU device (0 = first GPU)
)

Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/neu-surface-defect-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=defect_detection_v1, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=20, perspective=0

In [ ]:
metrics = model.val()
print(f"mAP50: {metrics.box.map50}")
print(f"mAP50-95: {metrics.box.map}")

Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 381.2±235.1 MB/s, size: 7.0 KB)
val: Scanning /content/neu-surface-defect-1/valid/labels.cache... 360 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 360/360 665.2Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 23/23 3.5it/s 6.6s
                   all        360        830      0.725      0.727      0.762      0.434
               crazing         56        126      0.478      0.381      0.395      0.128
             inclusion         78        212      0.801      0.802      0.837      0.432
               patches         71        173      0.805      0.861      0.889      0.585
        pitted_surface         53         79      0.779       0.81      0.881      0.592
       rolled-in_scale         68  

In [ ]:
from google.colab import files
import shutil

In [ ]:
shutil.make_archive("my_folder1", 'zip', "quality_control")


'/content/my_folder1.zip'

In [ ]:
!yolo export model=/content/quality_control/defect_detection_v1/weights/best.pt format=engine half=True


WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 72 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '/content/quality_control/defect_detection_v1/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 10, 8400) (21.5 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 9.0s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.1 opset 20...
ONNX: slimming with onnxslim 0.1.73...
ONNX: export success ✅ 11.1s, saved as '/content/quality_control/defect_detection_v1/weights/best.on

In [ ]:
!yolo predict model=/content/best.pt source='/content/test.jpg' imgsz=640 save=True project='/content'

Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
Model summary (fused): 72 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs

image 1/1 /content/test.jpg: 384x640 1 pitted_surface, 391.8ms
Speed: 5.3ms preprocess, 391.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict
